<a href="https://colab.research.google.com/github/rohanreddyp298/Democratizing-Database-Access---Text-to-SQL/blob/main/Democratizing_Database_Access_with_Text_to_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Project: Democratizing Database Access with Text-to-SQL

**Student:** Rohan
**Course:** DAMG 7370 - Designing Advanced Data Architectures
**Date:** February 9, 2026

## 1. Executive Summary & Objective
In the modern enterprise, the gap between data availability and data accessibility is a critical bottleneck. While organizations possess vast SQL databases, extracting insights requires specialized technical skills.

**The Goal:** This assignment aims to bridge that gap by fine-tuning a **Small Language Model (SLM)** to act as a specialized "Text-to-SQL" interface.

**The Approach:**
* **Model:** `deepseek-coder-1.3b-instruct` (chosen for its pre-trained code understanding).
* **Technique:** **QLoRA** (Quantized Low-Rank Adaptation) to efficiently fine-tune on a single T4 GPU.
* **Dataset:** `b-mc2/sql-create-context` (15,000 samples).
* **Target:** Achieve >40% Exact Match accuracy on unseen complex queries.

## 2. Environment Setup & Dependency Management
To run this pipeline on a standard Google Colab T4 GPU (16GB VRAM), we must leverage specific libraries for efficient training.

**Key Libraries:**
* `bitsandbytes`: Enables **4-bit quantization**, reducing model footprint from ~6GB to ~2GB VRAM.
* `peft`: Implements **LoRA**, allowing us to freeze the base model and only train a small percentage of adapter parameters.
* `trl`: Transformers Reinforcement Learning library, providing the `SFTTrainer` for supervised fine-tuning loops.

*Note: Specific versions are pinned below to ensure compatibility with Colab's CUDA drivers.*


## 3. Data Preparation & Engineering
A generic LLM cannot generate accurate SQL without context. If we ask "Show me the sales," it doesn't know if the table is named `sales`, `orders`, or `transactions`.

**Dataset Selection:**
We utilize the `b-mc2/sql-create-context` dataset, which provides a triplet of:
1.  **Question:** Natural language query.
2.  **Context:** The SQL Schema (CREATE TABLE statement).
3.  **Answer:** The correct SQL query.

**Preprocessing Strategy:**
We implement **Instruction Tuning** by formatting each sample into a strict prompt template. This forces the model to attend to the *Schema Context* before generating the *Response*.

$$\text{Prompt} = \text{Instruction} + \text{User Query} + \text{Schema Context} \rightarrow \text{SQL Output}$$

In [ ]:
# 1. Install Dependencies
# We unpin bitsandbytes to get the latest CUDA 12 support
# We keep trl pinned to 0.8.6 to ensure your code runs without syntax errors
!pip install -q -U bitsandbytes
!pip install -q -U triton
!pip install -q -U transformers==4.41.2 peft==0.11.1 datasets==2.19.1 trl==0.8.6 accelerate==0.30.1

import torch
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# 2. Dataset Selection & Preprocessing
print("Loading dataset...")
dataset = load_dataset("b-mc2/sql-create-context", split="train[:15000]")

# 3. Data Cleaning & Formatting
def format_instruction(sample):
    return f"""### Instruction:
You are a powerful SQL expert. Convert the following natural language question into a SQL query using the provided context.

### Question:
{sample['question']}

### Context:
{sample['context']}

### Response:
{sample['answer']}
"""

# Apply formatting
dataset = dataset.map(lambda x: {"text": format_instruction(x)})

# 4. Splitting (Train/Val/Test)
train_test = dataset.train_test_split(test_size=0.2, seed=42)
test_val = train_test['test'].train_test_split(test_size=0.5, seed=42)

dataset_dict = {
    'train': train_test['train'],
    'validation': test_val['train'],
    'test': test_val['test']
}

print(f"Data Shapes: Train: {len(dataset_dict['train'])}, Val: {len(dataset_dict['validation'])}, Test: {len(dataset_dict['test'])}")

## 4. Model Architecture & Quantization
We selected **DeepSeek-Coder-1.3B-Instruct** as our base model. Unlike general chat models (e.g., Llama-2-7B), DeepSeek is pre-trained on massive code repositories, giving it an inherent understanding of SQL syntax (`SELECT`, `GROUP BY`, `JOIN`).

**QLoRA Configuration:**
To fit this model into memory, we employ **4-bit Normal Float (NF4)** quantization.
* **Computation:** Matrix multiplications occur in float16.
* **Storage:** Weights are stored in 4-bit.
* **Rank (r):** We target **all linear layers** (`q_proj`, `k_proj`, `v_proj`, `o_proj`, etc.) to maximize learning capacity.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Model Selection: deepseek-coder-1.3b
# Justification: It's fast, efficient, and fits easily on Colab T4 while being capable of SQL logic.
model_id = "deepseek-ai/deepseek-coder-1.3b-instruct"

# Quantization Config (4-bit loading for efficiency)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load Model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False # Silence warnings for training

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token # Fix padding for Llama models
tokenizer.padding_side = "right"

## 5. Hyperparameter Optimization (Grid Search)
To meet the "Quality Score" requirement, we do not rely on default settings. We conducted a grid search across three distinct configurations to optimize for **Validation Loss**.

**Experimental Design:**
1.  **Config 1 (Aggressive):** High Learning Rate ($2e^{-4}$), Low Rank ($r=8$). Testing rapid convergence.
2.  **Config 2 (Deep):** Low Learning Rate ($2e^{-5}$), High Rank ($r=64$). Testing detailed feature extraction.
3.  **Config 3 (Balanced):** Mid-range settings.

*The code block below executes this search on a subset of data to identify the winner.*

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Define 3 configurations to test
hyperparams_list = [
    {"lr": 2e-4, "r": 8, "alpha": 16, "batch": 4},
    {"lr": 2e-5, "r": 64, "alpha": 128, "batch": 2},
    {"lr": 5e-5, "r": 16, "alpha": 32, "batch": 4}
]

results = {}

for i, params in enumerate(hyperparams_list):
    print(f"\n--- Testing Configuration {i+1} ---")

    # 1. PEFT Config (LoRA)
    peft_config = LoraConfig(
    r=params["r"],
    lora_alpha=params["alpha"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

    # 2. Training Arguments (Classic Stable Syntax)
    args = TrainingArguments(
        output_dir=f"./results_run_{i+1}",
        num_train_epochs=1,
        per_device_train_batch_size=params["batch"],
        gradient_accumulation_steps=4,
        learning_rate=params["lr"],
        logging_steps=10,
        evaluation_strategy="steps",  # Standard name for this version
        eval_steps=50,
        save_strategy="no",
        fp16=True,
        report_to="none"
    )

    # 3. Trainer
    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset_dict['train'].select(range(200)),
        eval_dataset=dataset_dict['validation'].select(range(50)),
        peft_config=peft_config,
        dataset_text_field="text", # Explicitly allowed in 0.8.6
        max_seq_length=1024,        # Explicitly allowed in 0.8.6
        tokenizer=tokenizer,
        args=args,
    )

    # Train
    trainer.train()

    # Log Final Eval Loss
    final_loss = trainer.evaluate()['eval_loss']
    results[f"Config {i+1}"] = final_loss
    print(f"Config {i+1} Final Loss: {final_loss}")

print("\nHyperparameter Search Results:", results)
best_config_index = min(results, key=results.get)
print(f"Best Configuration is: {best_config_index}")

## 6. Final Training Pipeline
Based on the Hyperparameter Search results, **Config 2 (Rank 64, LR 2e-5)** demonstrated the best stability and lowest loss.

We now execute the full training run on the complete dataset (15,000 examples).
* **Epochs:** 3
* **Batch Size:** 4 (with Gradient Accumulation steps = 2)
* **Callbacks:** `EarlyStoppingCallback` is implemented to prevent overfitting if validation loss increases.

Runtime:- 2hrs

In [ ]:
from transformers import EarlyStoppingCallback

# We use the parameters from the winning "Config 2"
# Rank (r) = 64, Alpha = 128
best_peft_config = LoraConfig(
    r=64,
    lora_alpha=128,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

final_args = TrainingArguments(
    output_dir="./final_model_sql",
    num_train_epochs=3,                     # Full training (3 epochs)
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,                     # Winning Learning Rate
    logging_steps=25,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=100,
    load_best_model_at_end=True,            # Rubric: Checkpointing
    report_to="tensorboard",                # Rubric: Logging
    save_total_limit=2
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['validation'],
    peft_config=best_peft_config,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=final_args,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)] # Rubric: Callbacks
)

print("Starting Final Training...")
trainer.train()
trainer.save_model("./final_best_model")
print("Training Complete!")

## 7. Model Evaluation & Error Analysis
We evaluate the model using **Exact Match Accuracy** on the unseen Test Set.

**Metric Definition:**
$$\text{Accuracy} = \frac{\text{Count(Generated SQL == Reference SQL)}}{\text{Total Test Samples}}$$

**Qualitative Analysis Plan:**
We will analyze specific failure modes to distinguish between:
* **Syntax Errors:** Invalid SQL (e.g., missing brackets).
* **Logic Errors:** Correct syntax but wrong intent (e.g., `AVG` instead of `SUM`).
* **Hallucinations:** Inventing columns not present in the schema context.

In [ ]:
import pandas as pd
from tqdm import tqdm

# 1. Switch Model to Evaluation Mode
model.eval()

def normalize_sql(sql_text):
    """Cleans up SQL for fairer comparison."""
    if not sql_text: return ""
    # Remove newlines and extra spaces
    sql_text = sql_text.replace("\n", " ").strip()
    # Lowercase everything for comparison (SELECT == select)
    sql_text = " ".join(sql_text.lower().split())
    # Remove trailing semicolons
    return sql_text.rstrip(";")

def generate_sql(query, context, model, tokenizer):
    prompt = f"""### Instruction:
You are a powerful SQL expert. Convert the following natural language question into a SQL query using the provided context.

### Question:
{query}

### Context:
{context}

### Response:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=200,          # Increased to prevent cut-offs
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1       # Slight penalty to stop loops
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # CRITICAL FIX: The model sometimes babbles after the answer.
    # We take the text AFTER "### Response:"
    raw_response = decoded.split("### Response:")[-1].strip()

    # We also cut it off if it starts hallucinating new sections like "###" or "Useful links"
    clean_response = raw_response.split("###")[0].split("\n\n")[0].strip()

    return clean_response

# 2. Run Inference on Test Set
# We increase the sample size to 50 to get a better statistical representation
test_samples = dataset_dict['test'].select(range(50))
results_list = []

print("Running Inference on Test Set (Improved)...")
for sample in tqdm(test_samples):
    generated_sql = generate_sql(sample['question'], sample['context'], model, tokenizer)

    norm_gen = normalize_sql(generated_sql)
    norm_exp = normalize_sql(sample['answer'])

    # Strict Match: Exact string match
    is_exact = norm_gen == norm_exp

    # Soft Match: Did we at least get the right table and columns? (Good for analysis)
    is_partial = (sample['answer'].split()[1] in generated_sql)

    results_list.append({
        "Question": sample['question'],
        "Expected": sample['answer'],
        "Generated": generated_sql,
        "Exact Match": is_exact
    })

# 3. Calculate Accuracy
df = pd.DataFrame(results_list)
accuracy = df["Exact Match"].mean() * 100
print(f"\nImproved Accuracy: {accuracy:.2f}%")

# 4. Display Analysis
print("\n--- ✅ SUCCESS EXAMPLES ---")
print(df[df["Exact Match"] == True][["Question", "Generated"]].head(2).to_string(index=False))

print("\n--- ❌ ERROR ANALYSIS (For Report) ---")
# Show close calls where we missed by a little bit
errors = df[df["Exact Match"] == False].head(3)
for i, row in errors.iterrows():
    print(f"\nQ: {row['Question']}")
    print(f"Exp: {row['Expected']}")
    print(f"Gen: {row['Generated']}")